# <div align="center" style="color: #ff5733;">TSA Closure Second Notification</div>

In [1]:
# Import Libraries   

import pandas as pd  
import numpy as np  

from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

# Ask Balasubramani to refresh risk_credit_mis.tsa_account_closure table.
**Check if that is refreshed when Balasubramani confirms it**

In [2]:
sq = """select Run_Date, count(1)cnt from risk_credit_mis.tsa_account_closure group by 1 order by 1 desc"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df.head(1)

Job ID 7bdaaa7d-c335-4674-95d7-7e75ef3a67ae successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Run_Date,cnt
0,2024-03-19,534768


The date should show yesterday's date

# Read the data from the first sheet for Type1 and Type2 data provided by Marvin

In [3]:
d1 = pd.read_excel(r"D:\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\Jan24 Batch & Feb24 Ongoing Batch\Type 1 & 2.xlsx")
d1.shape

(7403, 2)

In [4]:
print(f"The unique count of customer_id is:\t{len(d1['Customer_Id'].unique())}")
d1.head()


The unique count of customer_id is:	7403


,Customer_Id,ACTION_TYPE
0,2316535,TYPE 1
1,2333395,TYPE 1
2,2313053,TYPE 1
3,2349183,TYPE 1
4,2330303,TYPE 1


# Read the data from the second sheet for Type3 data provided by Marvin

In [5]:
d2 = pd.read_excel(r"D:\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\Jan24 Batch & Feb24 Ongoing Batch\Type 3.xlsx")
d2.shape

(3193, 2)

In [6]:
print(f"The unique count of customer_id is:\t{len(d2['Customer_Id'].unique())}")
d2.head()

The unique count of customer_id is:	3193


,Customer_Id,ACTION_TYPE
0,1541174,TYPE 3
1,2004612,TYPE 3
2,2072344,TYPE 3
3,2106415,TYPE 3
4,2067352,TYPE 3


In [7]:
result = pd.concat([d1, d2], axis=0, ignore_index=True)
result.shape

(10596, 2)

In [8]:
result.ACTION_TYPE.value_counts()

ACTION_TYPE
TYPE 1    7389
TYPE 3    3193
TYPE 2      14
Name: count, dtype: int64

In [9]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10596 entries, 0 to 10595
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  10596 non-null  int64 
 1   ACTION_TYPE  10596 non-null  object
dtypes: int64(1), object(1)
memory usage: 165.7+ KB


In [10]:
result['Customer_Id'] = result['Customer_Id'].astype(str)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10596 entries, 0 to 10595
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  10596 non-null  object
 1   ACTION_TYPE  10596 non-null  object
dtypes: object(2)
memory usage: 165.7+ KB


In [11]:
a = tuple(result['Customer_Id'])
a[:5]

('2316535', '2333395', '2313053', '2349183', '2330303')

The date should be D-1? If the date is not D-1 then ask Bala to refresh the table.

Step1: At line 11 change the date to D-1<br><br>
Step2: At line around 36 (and (TSA_Opening_date < date '2023-10-18')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step3: At line around 45 (WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-10-18')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step4: At line 58 (and TSA_Opening_date < date '2023-06-18'  and (Last_TSA_Credit_Date < date '2023-06-18' and Last_TSA_Debit_Date < date '2023-06-18' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-06-18')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) <br><br>
Step5: At line 72 (Run_Date = date '2024-01-09'###DATE change here to D-1) Change the date to D-1<br><br>

First Notification we change all the steps <br><br>
Second and Third Notification we only change Step1 and Step5
For Second Notificaiton and Third Notificaiton the customre id should be from the file sent by Marvin. <br><br>

In [12]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '2024-03-19'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
),
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select distinct
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Deposit_Balance,a.Active_Term_Deposit_flag,a.Last_TSA_Credit_Date,a.Last_TSA_Debit_Date,
  a.ARI_Flag,a.Active_Loan,a.Total_Loan_Outstanding,  case when b.customerId is not null then 1 else 0 end as Approved_Loans,
  a.Pending_Loan_Applications,a.Hard_Reject_Loans_App,a.Soft_Reject_Loans_App,a.Cancelled_Loans_App, extract(date from a.Last_Login_Date) as Last_Login_Date,
  a.Last_TSA_EOD_Balance_Date,a.Last_Group_Stash_Credit_Date,a.Group_Stash_Balance_Owner,a.Run_Date,a.Batch,a.Processing_Month,a.pending_loans_not_expired_not_exempt,a.expired_loans,
  a.pending_loans_lastDt,a.expired_loans_lastDt,
  CASE 
    WHEN tsa_status = 'Active' and (Last_TSA_Credit_Date = date '1970-01-01' and Last_Group_Stash_Credit_Date is null)
    and (TSA_Opening_date < date '2023-12-27')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)  THEN 'TYPE 1'
    and b.customerId is null  THEN 'TYPE 1'


    WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-12-27')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    and (date_trunc(TSA_Opening_date,day) = date_trunc(Last_Login_Date,day)) THEN 'TYPE 2'

    
    WHEN ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) < 1 )
            and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
            and (Active_Term_Deposit_flag = 0)
            and TSA_Opening_date < date '2023-08-29'                                         ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and (Last_TSA_Credit_Date < date '2023-08-29'                                     ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and Last_TSA_Debit_Date < date '2023-08-29'                                         ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-08-29')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and tsa_status = 'Active'
            and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
            and b.customerId is null  
    THEN 'TYPE 3'
    ELSE 'NO ACTION'
    END AS ACTION_TYPE
  from w_tsa A
  left join w_approved b
  on cast(a.Customer_Id as int64) = b.customerId
  left join core_raw.customer_accounts c
  on c.OFCUSTOMERID = a.Customer_Id and c.CRINTERDESC = 'Transactional Savings Account Inv_R' and c.OFISCLOSED = 'N'
  where
  Run_Date = date '2024-03-19'###DATE change here to D-1
)
"""


In [13]:
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 76be7950-9469-41a4-8389-22102d3012c2 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [14]:
pd.set_option('display.max_columns', None)
df.sample(10)

,Customer_Id,TSA_Opening_date,TSA_Account_Number,OfblockAmount,TSA_Status,TSA_Balance,Solo_Stash_Balance,Group_Stash_Balance,Term_Deposit_Balance,Active_Term_Deposit_flag,Last_TSA_Credit_Date,Last_TSA_Debit_Date,ARI_Flag,Active_Loan,Total_Loan_Outstanding,Approved_Loans,Pending_Loan_Applications,Hard_Reject_Loans_App,Soft_Reject_Loans_App,Cancelled_Loans_App,Last_Login_Date,Last_TSA_EOD_Balance_Date,Last_Group_Stash_Credit_Date,Group_Stash_Balance_Owner,Run_Date,Batch,Processing_Month,pending_loans_not_expired_not_exempt,expired_loans,pending_loans_lastDt,expired_loans_lastDt,ACTION_TYPE
6658,2321001,2023-12-02,60823210010001,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2023-12-06,NaT,NaT,NaN,2024-03-19,Batch_1,March,0,1,NaT,2024-03-02,TYPE 1
2488,1492495,2022-07-25,60814924950003,0.0,Active,0.53,0.0,0.0,0.0,0,2023-07-31,2023-07-31,N,0,0.0,0,0,0,0,0,2023-07-05,2024-03-19,NaT,NaN,2024-03-19,Batch_1,March,0,0,NaT,NaT,TYPE 3
6572,2319372,2023-12-02,60823193720002,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2023-12-03,NaT,NaT,NaN,2024-03-19,Batch_1,March,0,1,NaT,2024-03-03,TYPE 1
1219,1468914,2022-07-16,60814689140001,0.0,Active,0.22,0.0,0.0,0.0,0,2023-06-30,2023-06-30,N,0,0.0,0,0,0,0,0,2023-06-02,2024-03-19,NaT,NaN,2024-03-19,Batch_1,March,0,0,NaT,NaT,TYPE 3
4833,2079255,2023-06-05,60820792550001,0.0,Active,0.05,0.0,0.0,0.0,0,2023-07-31,2023-07-10,N,0,0.0,0,0,0,0,0,2023-07-10,2024-03-19,NaT,NaN,2024-03-19,Batch_1,March,0,0,NaT,NaT,TYPE 3
3687,2317438,2023-12-18,60823174380005,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2023-12-18,NaT,NaT,NaN,2024-03-19,Batch_1,March,0,0,NaT,NaT,TYPE 1
10345,1944964,2023-03-16,60819449640004,0.0,Active,0.06,0.0,0.0,0.0,0,2023-06-30,2023-06-30,N,0,0.0,0,0,0,0,0,2023-06-30,2024-03-19,NaT,NaN,2024-03-19,Batch_1,March,0,0,NaT,NaT,TYPE 3
4124,2093407,2023-06-14,60820934070004,0.0,Active,0.08,0.0,0.0,0.0,0,2023-07-31,2023-07-31,N,0,0.0,0,0,1,0,0,2023-10-02,2024-03-19,NaT,NaN,2024-03-19,Batch_1,March,0,0,NaT,NaT,TYPE 3
663,2119680,2023-07-05,60821196800009,0.0,Active,0.00,0.0,0.0,0.0,0,2023-07-05,2023-07-05,N,0,0.0,0,0,0,0,0,2023-09-01,NaT,NaT,NaN,2024-03-19,Batch_1,March,0,0,NaT,NaT,TYPE 3
3739,2325424,2023-12-06,60823254240006,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2023-12-06,NaT,NaT,NaN,2024-03-19,Batch_1,March,0,1,NaT,2024-03-06,TYPE 1


In [15]:
df.shape

(10587, 32)

In [16]:
df['ACTION_TYPE'].value_counts()

ACTION_TYPE
TYPE 1       7312
TYPE 3       2984
NO ACTION     278
TYPE 2         13
Name: count, dtype: int64

In [17]:
df.to_excel(r"D:\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\Jan24 Batch & Feb24 Ongoing Batch\20240320_TSA_Account_Closure_2ndtNotification.xlsx", sheet_name='TSA_Closure_2ndNotification', index = False)